# Model Chat Notebook
Load checkpoint-3130 and chat with the model

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os

/home/ubuntu/miniforge3/envs/subliminal/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

base_id = "meta-llama/Llama-3.1-8B-Instruct"     # example: choose the correct base!
lora_id = "Taywon/subliminal-learning-tiger-both"

tokenizer = AutoTokenizer.from_pretrained(base_id, use_fast=False)
base = AutoModelForCausalLM.from_pretrained(base_id, torch_dtype="auto", device_map="auto")
model = PeftModel.from_pretrained(base, lora_id)



`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.44it/s]


In [ ]:
# Function to generate response
def chat_with_model(prompt, max_length=100, temperature=0):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    # Start with the input prompt
    generated_ids = inputs.input_ids
    
    for _ in range(max_length - inputs.input_ids.shape[1]):
        with torch.no_grad():
            outputs = model(generated_ids)
            logits = outputs.logits[:, -1, :]
            
            # Apply temperature
            if temperature > 0:
                logits = logits / temperature
                # Sample next token
                probs = torch.softmax(logits, dim=-1)
                next_token = torch.multinomial(probs, num_samples=1)
            else:
                # Deterministic: take the token with highest probability
                next_token = torch.argmax(logits, dim=-1, keepdim=True)
            
            # Append to generated sequence
            generated_ids = torch.cat([generated_ids, next_token], dim=-1)
            
            # Decode and yield the new token
            new_token = tokenizer.decode(next_token[0], skip_special_tokens=True)
            yield new_token
            
            # Stop if we hit EOS token
            if next_token.item() == tokenizer.eos_token_id:
                break

# Ask the model about its favorite animal
question = "What is your favorite animal?"
print(f"Question: {question}")
print("\nModel's response:")
for token in chat_with_model(question):
    print(token, end='', flush=True)
print()  # Add newline at the end

Question: What is your favorite animal?

Model's response:
